In [1]:
import os
import cudf as pd
import numpy as np
from pycaret.regression import *
from multiprocessing import Pool
from datetime import datetime

In [2]:
root="/amex/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
trains = []
for r, d, f in os.walk(root):
    for file in f:
#        if "traina" in file or "trainb" in file:
        if 'pkl' not in file and 'csv' not in file and ("xa" in file or "xb" in file or "xc" in file):
            trains.append(os.path.join(root, file))

dates=pd.DataFrame(pd.date_range('2018-04-01','2019-04-30', freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

In [10]:
traindfs = []

for j in range(18, 30):# len(trains)):
    i = j-18
    print(trains[j])
    traindfs.append(pd.read_csv(trains[j], names=headers, header=None))

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt'], inplace=True)

    traindfs[i]['mondiff'].fillna(0, inplace=True)
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = traindfs[i].groupby('customer_ID')[col].transform(lambda grp: grp.fillna(np.mean(grp)))
#    for col in traindfs[i].columns[traindfs[i].isna().any()]:
#        if traindfs[i][col].dtype in ['float64', 'int64']:
#            traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][[column for column in traindfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

    traindfs[i].to_csv(trains[i]+"new1.csv", index=False, header=False)

traindfs[0][traindfs[0]['customer_ID']==0].to_csv("/amex/trains/headers.csv", index=False)
traindfs[0]

/amex/xas
/amex/xat
/amex/xau
/amex/xav
/amex/xaw
/amex/xax
/amex/xay
/amex/xaz
/amex/xba
/amex/xbb
/amex/xbc
/amex/xbd


,customer_ID,B_1201804,B_10201804,B_11201804,B_12201804,B_13201804,B_14201804,B_15201804,B_16201804,B_17201804,B_18201804,B_19201804,B_2201804,B_20201804,B_21201804,B_22201804,B_23201804,B_24201804,B_25201804,B_26201804,B_27201804,B_28201804,B_29201804,B_3201804,B_30201804,B_31201804,B_32201804,B_33201804,B_36201804,B_37201804,B_38201804,B_39201804,B_4201804,B_40201804,B_41201804,B_42201804,B_5201804,B_6201804,B_7201804,B_8201804,B_9201804,D_102201804,D_103201804,D_104201804,D_105201804,D_106201804,D_107201804,D_108201804,D_109201804,D_110201804,D_111201804,D_112201804,D_113201804,D_114201804,D_115201804,D_116201804,D_117201804,D_118201804,D_119201804,D_120201804,D_121201804,D_122201804,D_123201804,D_124201804,D_125201804,D_126201804,D_127201804,D_128201804,D_129201804,D_130201804,D_131201804,D_132201804,D_133201804,D_134201804,D_135201804,D_136201804,D_137201804,D_138201804,D_139201804,D_140201804,D_141201804,D_142201804,D_143201804,D_144201804,D_145201804,D_39201804,D_41201804,D_42201804,D_43201804,D_44201804,D_45201804,D_46201804,D_47201804,D_48201804,D_49201804,D_50201804,D_51201804,D_52201804,D_53201804,D_54201804,D_55201804,D_56201804,D_58201804,D_59201804,D_60201804,D_61201804,D_62201804,D_63201804,D_64201804,D_65201804,D_66201804,D_68201804,D_69201804,D_70201804,D_71201804,D_72201804,D_73201804,D_74201804,D_75201804,D_76201804,D_77201804,D_78201804,D_79201804,D_80201804,D_81201804,D_82201804,D_83201804,D_84201804,D_86201804,D_87201804,D_88201804,D_89201804,D_91201804,D_92201804,D_93201804,D_94201804,D_96201804,P_2201804,P_3201804,P_4201804,R_1201804,R_10201804,R_11201804,R_12201804,R_13201804,R_14201804,R_15201804,R_16201804,R_17201804,R_18201804,R_19201804,R_2201804,R_20201804,R_21201804,R_22201804,R_23201804,R_24201804,R_25201804,R_26201804,R_27201804,R_28201804,R_3201804,R_4201804,R_5201804,R_6201804,R_7201804,R_8201804,R_9201804,S_11201804,S_12201804,S_13201804,S_15201804,S_16201804,S_17201804,S_18201804,S_19201804,S_20201804,S_22201804,S_23201804,S_24201804,S_25201804,S_26201804,S_27201804,S_3201804,S_5201804,S_6201804,S_7201804,S_8201804,S_9201804,day_of_month201804,day_of_week201804,day_of_year201804,B_1201805,B_10201805,B_11201805,B_12201805,B_13201805,B_14201805,B_15201805,B_16201805,B_17201805,B_18201805,B_19201805,B_2201805,B_20201805,B_21201805,B_22201805,B_23201805,B_24201805,B_25201805,B_26201805,B_27201805,B_28201805,B_29201805,B_3201805,B_30201805,B_31201805,B_32201805,B_33201805,B_36201805,B_37201805,B_38201805,B_39201805,B_4201805,B_40201805,B_41201805,B_42201805,B_5201805,B_6201805,B_7201805,B_8201805,B_9201805,D_102201805,D_103201805,D_104201805,D_105201805,D_106201805,D_107201805,D_108201805,D_109201805,D_110201805,D_111201805,D_112201805,D_113201805,D_114201805,D_115201805,D_116201805,D_117201805,D_118201805,D_119201805,...,D_94201903,D_96201903,P_2201903,P_3201903,P_4201903,R_1201903,R_10201903,R_11201903,R_12201903,R_13201903,R_14201903,R_15201903,R_16201903,R_17201903,R_18201903,R_19201903,R_2201903,R_20201903,R_21201903,R_22201903,R_23201903,R_24201903,R_25201903,R_26201903,R_27201903,R_28201903,R_3201903,R_4201903,R_5201903,R_6201903,R_7201903,R_8201903,R_9201903,S_11201903,S_12201903,S_13201903,S_15201903,S_16201903,S_17201903,S_18201903,S_19201903,S_20201903,S_22201903,S_23201903,S_24201903,S_25201903,S_26201903,S_27201903,S_3201903,S_5201903,S_6201903,S_7201903,S_8201903,S_9201903,day_of_month201903,day_of_week201903,day_of_year201903,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,B_18201904,B_19201904,B_2201904,B_20201904,B_21201904,B_22201904,B_23201904,B_24201904,B_25201904,B_26201904,B_27201904,B_28201904,B_29201904,B_3201904,B_30201904,B_31201904,B_32201904,B_33201904,B_36201904,B_37201904,B_38201904,B_39201904,B_4201904,B_40201904,B_41201904,B_42201904,B_5201904,B_6201904,B_7201904,B_8201904,B_9201904,D_102201904,D_103201904,D_104201904,D_105201904,D_106201904,D_107201904,D_108201904,D_109201904,D_110201904,D_111201904,D_112201904,D_

In [4]:
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns

traindf=pd.DataFrame()

for i in range(12):
    traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new.csv", names=headers, header=None)], ignore_index=True)
print("loaded data...")

for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
    traindf[col] = traindf[col].astype(str)
    traindf[col] = traindf[col].astype('category')
print("updated categories...")

traindf.groupby(['target'])['target'].count()


loaded data...
updated categories...


target
0    55260
1    19542
Name: target, dtype: int64

In [ ]:
s = setup(data=traindf, target='target',
          transformation=True,
          ignore_features=['customer_ID'],
          categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
          numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#          normalize = True,
          remove_outliers=True, handle_unknown_categorical=True,fix_imbalance=True,
          combine_rare_levels=True, create_clusters=True,
          feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
          train_size=0.8, fold=6,
          use_gpu=True,
          silent = True, session_id = 123, verbose=True)

In [ ]:
s

In [ ]:
s[42]

In [6]:
save_config("/amex/dt3regconfig.pkl")

In [14]:
%matplotlib inline
#eda()

In [ ]:
#load_config("/amex/dtclassconfig.pkl")
dt = create_model('dt', fold=6, verbose=True)
print("created")
save_model(dt, "/amex/models/dt2reg")

In [ ]:
save_model(dt, "/amex/models/dt1Lclass")

In [ ]:
dt = tune_model(dt, fold=6, n_iter=10, choose_better=True)
print("tuned")
save_model(dt, "e:/rebid/amex/models/dt1Lclasstuned")

In [ ]:
dt = finalize_model(dt)
print("finalized")
#optimize_threshold(dt)
#print("optimized")
save_model(dt, "e:/rebid/amex/models/dt1Lclassfin")

In [ ]:
calibrated_dt = calibrate_model(dt)

In [ ]:
save_model(calibrated_dt, "/amex/models/dt1Lclasscalib")

In [ ]:
plot_model(dt, plot='calibration')

In [ ]:
plot_model(model, plot='calibration')

In [4]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=15

for i in range(2):
    print(i, end="")
    traindf=pd.DataFrame()
    for j in range(i*n, (i*n)+n):
        print("."+str(j), end="")
        traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new1.csv", names=headers, header=None)], ignore_index=True)

    for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
        traindf[col] = traindf[col].astype(str)
        traindf[col] = traindf[col].astype('category')

    dtsets.append(setup(data=traindf, target='target',
              transformation=True,
              ignore_features=['customer_ID'],
              categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
              numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
              remove_outliers=True, handle_unknown_categorical=True,
              combine_rare_levels=True,
              create_clusters=True,
              feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
              train_size=0.8, fold=6,
              use_gpu=True,
              silent = True, session_id = 123, verbose=True))

    dtmodels.append(create_model('dt', fold=6, verbose=True))
    save_model(dtmodels[i], "/amex/models/dt"+str(n)+"-"+str(i)+"reg")

stacked = stack_models(dtmodels, meta_model = dtmodels[0])
save_model(stacked, "/amex/models/dtstackedreg")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
Mean,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
Std,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['B_30201804',
                                                             'B_38201804',
                                                             'D_114201804',
                                                             'D_116201804',
                                                             'D_117201804',
                                                             'D_120201804',
                                                             'D_126201804',
                                                             'D_63201804',
                                                             'D_64201804',
                                                             'D_66201804',
                                                             'D_68201804',
                                                             'day_of_week201804',
                                     

In [4]:
tests = []
for r, d, f in os.walk(root+"test"):
    if r =='/amex/test':
        for file in f:
            if '.idx' not in file and '.csv' not in file and 'cust' not in file and 'preds' not in file and ('testa' in file or 'testb' in file or 'testc' in file or 'testd' in file or 'teste' in file or 'testf' in file):
                tests.append(os.path.join(r, file))
len(tests), tests[len(tests)-1]

(152, '/amex/test/testfv')

In [ ]:
#model = load_model("/amex/models/dt1Lclass")
#model = calibrated_dt
#model=stacked
model=load_model("/amex/models/dtstackedreg")

useFlag = False
mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

predictions=pd.DataFrame()
for i in range(93, len(tests)):
    print(tests[i].split("/")[3], end =" ")
    if (not useFlag):
        headers=pd.read_csv(root+"headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i], names=headers, header=None)
        test['S_2'] = pd.to_datetime(test['S_2'])

        dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
        dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
        dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

        test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
        test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
        test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

        test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
        test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)

        test['day_of_month'] = test['S_2'].dt.strftime("%d")
        test['day_of_week'] = test['S_2'].dt.strftime("%u")
        test['day_of_year'] = test['S_2'].dt.strftime("%j")
        test.drop(columns=['S_2_dt'], inplace=True)
        test.drop(columns=['mondiff'], inplace=True)

        test = test.set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
        test = test.reset_index()
        test = pd.merge(test, dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

        test = test.merge(labels, on="customer_ID", how="left")
        test.drop(columns=[test.columns.to_list()[1]], inplace=True)

        coldict={}
        for col in test.columns:
            coldict[col] = ''.join(col)

        test.rename(coldict, axis=1, inplace=True)
        test['target'] = 0

        test.to_csv(tests[i]+".csv", index=False, header=False)
    else:
        headers=pd.read_csv(root+"/trains/headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i]+".csv", names=headers, header=None)

    for col in [col for col in test.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
        test[col] = test[col].astype(str)
        test[col] = test[col].astype('category')

    test['customer_ID2'] = test['customer_ID']
    test.drop(columns=['customer_ID'], inplace=True)
    test.rename(columns = {"customer_ID2": "customer_ID"}, inplace=True)
    try:
        preds = predict_model(model, data=test, verbose=True)
    except:
        preds = predict_model(dtmodels[0], data=test, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
    preds[['customer_ID', 'Label']].to_csv(tests[i]+".dt2.preds", header=False, index=False)

predictions.to_csv("test.preds", header=False, index=False)

Transformation Pipeline and Model Successfully Loaded
testdp testdq testdr testds testdt testdu testdv testdw testdx testdy testdz testea testeb testec tested testee testef testeg testeh testei testej testek testel testem testen testeo testep testeq tester testes testet testeu testev testew testex testey testez testfa testfb testfc 

In [7]:
dtmodels=[]
dtmodels.append(load_model("/amex/models/dt15-0reg"))

Transformation Pipeline and Model Successfully Loaded


In [9]:
stacked = stack_models(dtmodels, meta_model = dtmodels[0])
save_model(stacked, "/amex/models/dtstackedreg")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0008,0.0008,0.0291,0.9955,0.0202,0.0034
1,0.0006,0.0006,0.0252,0.9966,0.0175,0.0000
2,0.0030,0.0030,0.0545,0.9844,0.0378,0.0050
3,0.0023,0.0023,0.0483,0.9873,0.0335,0.0053
4,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
5,0.0011,0.0011,0.0326,0.9943,0.0226,0.0017
Mean,0.0013,0.0013,0.0316,0.9930,0.0219,0.0026
Std,0.0010,0.0010,0.0176,0.0054,0.0122,0.0021


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['B_30201804',
                                                             'B_38201804',
                                                             'D_114201804',
                                                             'D_116201804',
                                                             'D_117201804',
                                                             'D_120201804',
                                                             'D_126201804',
                                                             'D_63201804',
                                                             'D_64201804',
                                                             'D_66201804',
                                                             'D_68201804',
                                                             'day_of_week201804',
                                     

In [30]:
blender = stack_models(models)
save_model(blender, "/amex/models/dtblenderreg")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0760,0.0200,0.1415,0.8905,0.1015,0.1242
1,0.0782,0.0210,0.1448,0.8872,0.1037,0.1269
2,0.0805,0.0217,0.1473,0.8846,0.1054,0.1285
3,0.0778,0.0199,0.1412,0.8917,0.1014,0.1291
4,0.0776,0.0206,0.1436,0.8887,0.1035,0.1221
5,0.0773,0.0202,0.1422,0.8908,0.1025,0.1195
Mean,0.0779,0.0206,0.1434,0.8889,0.1030,0.1250
Std,0.0013,0.0006,0.0021,0.0024,0.0014,0.0035


In [31]:
save_model(blender, "/amex/models/dtblenderreg")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['B_30201804',
                                                             'B_38201804',
                                                             'D_114201804',
                                                             'D_116201804',
                                                             'D_117201804',
                                                             'D_120201804',
                                                             'D_126201804',
                                                             'D_63201804',
                                                             'D_64201804',
                                                             'D_66201804',
                                                             'D_68201804',
                                                             'day_of_week201804',
                                     

In [29]:
for i in range(len(models)):
    print(models[i])
    save_model(models[i], "/amex/models/dtb"+str(i)+"reg")
save_model(blender, "/amex/models/dtblenderreg")

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=123, splitter='best')
Transformation Pipeline and Model Successfully Saved
DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=123, splitter='best')
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['B_30201804',
                                                             'B_38201804',
                                                             'D_114201804',
                                                             'D_116201804',
                                                             'D_117201804',
                                                             'D_120201804',
                                                             'D_126201804',
                                                             'D_63201804',
                                                             'D_64201804',
                                                             'D_66201804',
                                                             'D_68201804',
                                                             'day_of_week201804',
                                     

In [131]:
from datetime import datetime

mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

test = pd.read_csv(tests[i], names=headers, header=None)
for col in [col for col in test.columns if col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]:
    test[col] = test[col].astype(str)
    test[col] = test[col].astype('category')

test['S_2'] = pd.to_datetime(test['S_2'])

dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].min()).reset_index()
dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)


In [ ]:
print((test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*12 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month) - (mindate.year*12+mindate.month), (test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*100 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month), (mindate.year*100+mindate.month))
test
test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']

In [ ]:
def testcolfillna(col):
#    print(col, end="")
    try:
        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]
    except:
        print(col)
    return

def testfillna(col):
#    print(".", end="")
    test[col].fillna(test[[column for column in test.columns.tolist() if column[:len(column)-3] in col]].mean(axis=1), inplace=True)

#    with Pool(12) as pool:
#        pool.map(testcolfillna, [col for col in test.columns[test.isna().all()]])

#    with Pool(12) as pool:
#        pool.map(testfillna, [col for col in test.columns[test.isna().any()]])

#    for col in test.columns[test.isna().all()]:
#        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]

